In [1]:
import os,sys
import time,datetime
import GPy
import GPyOpt
import numpy as np
from scipy.stats import bernoulli, norm
import pandas as pd

In [2]:
%run ../tools/TrainTest.py
%run ../tools/PG1GM.py

In [3]:
trial_num = 5
tune_trial_num = 100
snapshot_path = '../result/snapshot/complement'
snapshot_form = 'PG1-hyper5-EI5000w500-split{0}-{1}'
peer_review_csv = '../dataset/open_peer_review/peer_review/peer_review_forPG3_suffled.csv'

# functions
def timeStamp():
    # return time as str
    todaydetail = datetime.datetime.today()
    return todaydetail.strftime("%Y%m%d%H%M%S")

def saveGPyOpt(myBopt,fold_num):
    filename = snapshot_form.format(fold_num,timeStamp())
    path = os.path.join(snapshot_path, filename)
    np.savez_compressed(path, myBopt.X, myBopt.Y)
    print("Save: {}".format(path))

In [4]:
# preprocess train&test dataset
rDF = pd.read_csv(peer_review_csv)
np.random.seed(12345678)
shuffled_pattern = [np.random.permutation(len(rDF)) for i in xrange(trial_num)]
df = rDF.take(shuffled_pattern[0]).reset_index(drop=True)
train, val, test = splitDataset523(df)

In [5]:
pg1 = PG1GM(train,val,test)
pg1.rmseForVal([2.40841889, 6.77462461, 79.11369152, 85.80229935, 21.78680711])

0.9043816311159756

In [6]:
pg1.rmseForTest([2.40841889, 6.77462461, 79.11369152, 85.80229935, 21.78680711])

0.8922516497202919

making aucForVal(self,hyper_list)

In [7]:
stan_fit = pg1.fit([2.40841889, 6.77462461, 79.11369152, 85.80229935, 21.78680711])

In [8]:
stan_fit

Inference for Stan model: anon_model_8a16fa5f1a4901cc869fc37b72c4603e.
4 chains, each with iter=5000; warmup=500; thin=1; 
post-warmup draws per chain=4500, total post-warmup draws=18000.

                   mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
ability[0]         2.46  1.0e-3   0.14   2.18   2.36   2.46   2.55   2.73  18000    1.0
ability[1]         2.41  1.1e-3   0.15   2.12   2.31   2.41   2.51   2.69  18000    1.0
ability[2]         2.44  1.1e-3   0.15   2.15   2.34   2.44   2.54   2.73  18000    1.0
ability[3]         2.49  8.7e-4   0.12   2.26   2.41   2.49   2.57   2.72  18000    1.0
ability[4]          2.4  1.1e-3   0.15   2.11    2.3    2.4    2.5   2.68  18000    1.0
ability[5]          2.5  1.1e-3   0.14   2.22    2.4    2.5   2.59   2.77  18000    1.0
ability[6]         2.45  1.1e-3   0.14   2.18   2.36   2.45   2.55   2.74  18000    1.0
ability[7]         2.41  1.1e-3   0.15   2.12   2.31   2.41   2.51    2.7  18000    1.0
ability[8]         

In [9]:
eap_value = stan_fit.summary()['summary'][:,0]

In [10]:
split = pg1.userNum
ability = eap_value[0:split]
reliability = eap_value[split:split*2]
bias = eap_value[split*2:split*3]

In [11]:
sender = pg1.senderVal
receiver = pg1.receiverVal

In [12]:
sender[0], receiver[0]

(96, 12)

In [13]:
val

sender_id  receiver_id  corrected  value
0            96           12          1      3
1           247          151          0      3
2            30          192          0      3
3            21           67          0      4
4             3          227          1      3
5            57           34          0      4
6           165          200          1      1
7           170          140          1      3
8           247          376          1      3
9            34          269          1      1
10           34           55          1      3
11          170           14          1      3
12          165          215          0      2
13          166          352          1      4
14           14           60          1      3
15           52          134          0      4
16          132          282          0      3
17          165          209          1      1
18          158           58          1      1
19           14           23          1      3
20          121          165          0      4
21           34          130          1      1
22          111          165          1      2
23          247           58          0      4
24          247          245          1      3
25          140           58          1      2
26           34          266          1      1
27           63           53          1      3
28          166          165          0      4
29          239          236          1      4
...         ...          ...        ...    ...
1020        154          133          1      2
1021         34          185          1      2
1022         34          110          1      4
1023          3          221          1      3
1024        101           34          1      2
1025        132           58          1      2
1026        121           58          1      2
1027         45          292          0      1
1028         52          165          0      4
1029        165          224          1      3
1030          3           53          1      3
1031        165          194          1      1
1032         57           12          1      3
1033        247          207          0      3
1034        165          247          1      2
1035        165          210          1      2
1036        132          165          0      3
1037        132          402          1      2
1038        129          210          1      2
1039        176          165          0      4
1040         52           25          1      3
1041        101          192          1      2
1042         97          376          1      3
1043        101           59          1      1
1044        154          165          0      3
1045        101            3          1      3
1046        101           30          0      2
1047        123          195          1      2
1048         34          233          1      1
1049        101          195          1      3

[1050 rows x 4 columns]

In [14]:
sender

array([ 96, 247,  30, ..., 123,  34, 101])

In [15]:
valueEst = ability[receiver] + bias[sender]

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(pg1.valueVal, valueEst))

In [17]:
rmse

0.9043937992249399

In [19]:
len(pg1.valueVal), len(valueEst)

(1050, 1050)